# Training and Evaluation of Models


## Part III: Stepwise Methods :  AIC Criterion


### Processed Data from Exercise 6 for Further Analysis

In [3]:
dane <- read.csv2("C:/Users/Dell/OneDrive/Pulpit/studia/IV SEM/modelowanie/cwl6/weather.csv")
dane_nowe = dane[c( 'LAT', 'LON', 'ALTITUDE', 'RECORD.LENGTH','MEAN.ANNUAL.AIR.TEMP', 'MEAN.MONTHLY.MAX.TEMP', 'MEAN.MONTHLY.MIN.TEMP', 'MEAN.ANNUAL.WIND.SPEED', 'MEAN.CLOUD.COVER','MEAN.ANNUAL.SUNSHINE', 'MEAN.ANNUAL.RAINFALL', 'MAX.MONTHLY.WIND.SPEED', 'MAX.AIR.TEMP', 'MAX.WIND.SPEED', 'MAX.RAINFALL', 'MIN.AIR.TEMP', 'MEAN.RANGE.AIR.TEMP')]


sapply(dane_nowe, function(x) sum(x == ""))
dane_nowe[dane_nowe == ""] <- NA
liczba_wierszy_przed <- nrow(dane_nowe)
dane_czyste <- na.omit(dane_nowe)
liczba_wierszy_po <- nrow(dane_czyste)
liczba_usunietych_wierszy <- liczba_wierszy_przed - liczba_wierszy_po


split = sample.split(dane_czyste, SplitRatio = 0.3 , group = NULL)
dane_uczace = subset(dane_czyste, split == FALSE)
dane_testowe = subset(dane_czyste, split == TRUE)

vec_train = as.vector(dane_uczace['MEAN.ANNUAL.RAINFALL'])

vec_test = as.vector(dane_testowe['MEAN.ANNUAL.RAINFALL'])


matrix_test = data.matrix(dane_testowe[colnames(dane_testowe) != 'MEAN.ANNUAL.RAINFALL']) #bez zmiennej objaśnianej
matrix_train = data.matrix(dane_uczace[colnames(dane_uczace) != 'MEAN.ANNUAL.RAINFALL'])
korelacja <-  cor(dane_czyste)



LAT                    LON               ALTITUDE 
                     0                      0                      0 
         RECORD.LENGTH   MEAN.ANNUAL.AIR.TEMP  MEAN.MONTHLY.MAX.TEMP 
                     0                     NA                     NA 
 MEAN.MONTHLY.MIN.TEMP MEAN.ANNUAL.WIND.SPEED       MEAN.CLOUD.COVER 
                    NA                     NA                     NA 
  MEAN.ANNUAL.SUNSHINE   MEAN.ANNUAL.RAINFALL MAX.MONTHLY.WIND.SPEED 
                    NA                     NA                     NA 
          MAX.AIR.TEMP         MAX.WIND.SPEED           MAX.RAINFALL 
                    NA                     NA                     NA 
          MIN.AIR.TEMP    MEAN.RANGE.AIR.TEMP 
                    NA                      0

### Nonlinear Regression Model for Annual Rainfall Prediction Using Altitude Features

This task involves creating a nonlinear regression model to predict mean annual rainfall based on three altitude-derived features: altitude, the logarithm of altitude, and the square of altitude. The model is trained on the training dataset, and the Root Mean Square Error (RMSE) is calculated to evaluate its fit to the observed data. Additionally, the model's predictions are tested on the test dataset using the predict() function, and the RMSE is calculated to assess the prediction accuracy.

In [4]:
model1<-lm(MEAN.ANNUAL.RAINFALL ~ ALTITUDE+log(ALTITUDE)+ALTITUDE^2, data = dane_uczace)
rmse1_tr <-sqrt(mean((model1$residuals^2)))

print(rmse1_tr)


[1] 132.2739


In [5]:
predict1 <- predict(model1, newdata = dane_testowe)
rmse2_test <- sqrt(mean((predict1 - dane_testowe$MEAN.ANNUAL.RAINFALL)^2))

print(rmse2_test)

[1] 162.8355


### Nonlinear Regression Model for Annual Rainfall Prediction Using Maximum Rainfall Features

This task involves creating a nonlinear regression model to predict mean annual rainfall based on three features derived from maximum rainfall: maximum rainfall, the logarithm of maximum rainfall, and the square of maximum rainfall. The model is trained on the training dataset, and the Root Mean Square Error (RMSE) is calculated to evaluate its fit to the observed data. The model's predictions are then applied to the test dataset using the predict() function, and the RMSE is calculated to assess the prediction accuracy on unseen data.

In [6]:
model2<-lm(MEAN.ANNUAL.RAINFALL ~ MAX.RAINFALL+log(MAX.RAINFALL)+MAX.RAINFALL^2, data = dane_uczace)
rmse3_tr <-sqrt(mean((model2$residuals^2)))

print(rmse3_tr)


[1] 116.4591


In [7]:
predict2 <- predict(model2, newdata = dane_testowe)
rmse4_test <- sqrt(mean((predict2 - dane_testowe$MEAN.ANNUAL.RAINFALL)^2))

print(rmse4_test)

[1] 114.7046


### Forward Selection for Model Building and Evaluation of Annual Rainfall Prediction



In [11]:
model3<-lm(MEAN.ANNUAL.RAINFALL ~ 1, data = dane_uczace)

scope1<-as.formula(MEAN.ANNUAL.RAINFALL ~ ALTITUDE + MAX.RAINFALL + MEAN.CLOUD.COVER + MEAN.ANNUAL.AIR.TEMP)
model4 <- step(model3, scope = scope1, direction = 'forward')
summary(model4)


Start:  AIC=1555.69
MEAN.ANNUAL.RAINFALL ~ 1

                       Df Sum of Sq     RSS    AIC
+ MAX.RAINFALL          1   4892980 2092251 1384.1
+ ALTITUDE              1   3937963 3047267 1438.2
+ MEAN.ANNUAL.AIR.TEMP  1   2847322 4137908 1482.3
+ MEAN.CLOUD.COVER      1    420682 6564548 1548.7
<none>                              6985231 1555.7

Step:  AIC=1384.09
MEAN.ANNUAL.RAINFALL ~ MAX.RAINFALL

                       Df Sum of Sq     RSS    AIC
+ MEAN.ANNUAL.AIR.TEMP  1    138857 1953393 1376.2
+ ALTITUDE              1     76184 2016067 1380.8
+ MEAN.CLOUD.COVER      1     31181 2061070 1383.9
<none>                              2092251 1384.1

Step:  AIC=1376.2
MEAN.ANNUAL.RAINFALL ~ MAX.RAINFALL + MEAN.ANNUAL.AIR.TEMP

                   Df Sum of Sq     RSS    AIC
<none>                          1953393 1376.2
+ MEAN.CLOUD.COVER  1    3698.9 1949694 1377.9
+ ALTITUDE          1    1633.4 1951760 1378.1



Call:
lm(formula = MEAN.ANNUAL.RAINFALL ~ MAX.RAINFALL + MEAN.ANNUAL.AIR.TEMP, 
    data = dane_uczace)

Residuals:
    Min      1Q  Median      3Q     Max 
-282.29  -75.55   -0.40   89.96  366.09 

Coefficients:
                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)           156.648    132.333   1.184  0.23851    
MAX.RAINFALL           22.343      1.779  12.557  < 2e-16 ***
MEAN.ANNUAL.AIR.TEMP  -29.579      9.343  -3.166  0.00189 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 117.7 on 141 degrees of freedom
Multiple R-squared:  0.7204,	Adjusted R-squared:  0.7164 
F-statistic: 181.6 on 2 and 141 DF,  p-value: < 2.2e-16


In [12]:
model5<-lm(MEAN.ANNUAL.RAINFALL~ MAX.RAINFALL + MEAN.ANNUAL.AIR.TEMP, data = dane_uczace)
rmse5_tr <-sqrt(mean((model5$residuals^2)))

print(rmse5_tr)


[1] 116.4699


In [13]:
predict3 <- predict(model5, newdata = dane_testowe)
rmse6_test <- sqrt(mean((predict3 - dane_testowe$MEAN.ANNUAL.RAINFALL)^2))

print(rmse6_test)

[1] 113.4446


### Backward Selection for Model Building and Evaluation of Annual Rainfall Prediction


In [14]:
model7<-lm(MEAN.ANNUAL.RAINFALL ~ ALTITUDE + MAX.RAINFALL + MEAN.CLOUD.COVER + MEAN.ANNUAL.AIR.TEMP, dane_uczace)
model8 <- step(model7, direction = 'backward')
summary(model8)


Start:  AIC=1379.8
MEAN.ANNUAL.RAINFALL ~ ALTITUDE + MAX.RAINFALL + MEAN.CLOUD.COVER + 
    MEAN.ANNUAL.AIR.TEMP

                       Df Sum of Sq     RSS    AIC
- ALTITUDE              1      1665 1949694 1377.9
- MEAN.CLOUD.COVER      1      3731 1951760 1378.1
<none>                              1948029 1379.8
- MEAN.ANNUAL.AIR.TEMP  1     53058 2001087 1381.7
- MAX.RAINFALL          1   1078949 3026978 1441.3

Step:  AIC=1377.93
MEAN.ANNUAL.RAINFALL ~ MAX.RAINFALL + MEAN.CLOUD.COVER + MEAN.ANNUAL.AIR.TEMP

                       Df Sum of Sq     RSS    AIC
- MEAN.CLOUD.COVER      1      3699 1953393 1376.2
<none>                              1949694 1377.9
- MEAN.ANNUAL.AIR.TEMP  1    111375 2061070 1383.9
- MAX.RAINFALL          1   2188061 4137756 1484.3

Step:  AIC=1376.2
MEAN.ANNUAL.RAINFALL ~ MAX.RAINFALL + MEAN.ANNUAL.AIR.TEMP

                       Df Sum of Sq     RSS    AIC
<none>                              1953393 1376.2
- MEAN.ANNUAL.AIR.TEMP  1    138857 2092251 1


Call:
lm(formula = MEAN.ANNUAL.RAINFALL ~ MAX.RAINFALL + MEAN.ANNUAL.AIR.TEMP, 
    data = dane_uczace)

Residuals:
    Min      1Q  Median      3Q     Max 
-282.29  -75.55   -0.40   89.96  366.09 

Coefficients:
                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)           156.648    132.333   1.184  0.23851    
MAX.RAINFALL           22.343      1.779  12.557  < 2e-16 ***
MEAN.ANNUAL.AIR.TEMP  -29.579      9.343  -3.166  0.00189 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 117.7 on 141 degrees of freedom
Multiple R-squared:  0.7204,	Adjusted R-squared:  0.7164 
F-statistic: 181.6 on 2 and 141 DF,  p-value: < 2.2e-16


In [15]:
model9<-lm(MEAN.ANNUAL.RAINFALL ~ MAX.RAINFALL +MEAN.ANNUAL.AIR.TEMP , data = dane_uczace)
rmse7_tr <-sqrt(mean((model9$residuals^2)))

print(rmse7_tr)


[1] 116.4699


In [16]:
predict4 <- predict(model9, newdata = dane_testowe)
rmse8_test <- sqrt(mean((predict4 - dane_testowe$MEAN.ANNUAL.RAINFALL)^2))

print(rmse8_test)

[1] 113.4446
